#### **Scraping website from most common website for scraping**

Website :
- *bookstoscrape.com*


#### Scrapy

In [13]:
import scrapy
from scrapy import cmdline
import schedule
import time

In [ ]:
# run scrapy in terminal CMD

# change directory to folder where 'spider' folder located
cd scrapy_project
cd folder_contain_scrapy_project

# create venv
python -m venv env

# install scrapy
pip install scrapy

# activate env
env\Scripts\activate.bat
env\Scripts\activate

# run script
!scrapy crawl books

In [24]:
!python -m venv env


In [12]:
def crawl():
  cmdline.execute('scrapy crawl scraper.py'.split())

In [ ]:
schedule.every(3).minutes.do(crawl)

# while True:
#   schedule.run_pending()
#   time.sleep(1)

KeyboardInterrupt: 

In [9]:
# function to run the spider
def crawl_wiki():
    cmdline.execute("scrapy runspider WikiSpider.py".split())

# schedule the spider to run every 30 seconds
schedule.every(30).seconds.do(crawl_wiki)

Every 30 seconds do crawl_wiki() (last run: [never], next run: 2024-11-29 21:18:12)

#### <hr>

#### BeautifulSoup (BS4)

In [95]:
# import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [3]:
# get url and send GET request
url1 = 'https://books.toscrape.com/'
url2 = 'https://books.toscrape.com/catalogue/page-1.html'

# method #1
get_url1 = requests.get(url1)
if get_url1.status_code == 200:
  print('Status Okey')
else: print('Status Failed')

# parse html content
soup1 = BeautifulSoup(get_url1.content,'html.parser')

# method #2
get_url2 = requests.get(url2)
get_url2 = get_url2.content

# parse html content
soup2 = BeautifulSoup(get_url2,'html.parser')

Status Okey


In [39]:
list_ordered = soup2.find('ol')
articles = list_ordered.find_all('article', class_='product_pod')

articles2 = soup1.find_all('h3')
book = soup1.find('a')['href']
requests.get(url1+book)
# articles

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" hre

##### Extract page of book per page
(take time because it loops each bookpage per page)

In [94]:
books_perpage = []
url1 = 'https://books.toscrape.com/catalogue/'

for h in range(1,51):
  outerpage_url = f'https://books.toscrape.com/catalogue/page-{h}.html'
  outerpage_request = requests.get(outerpage_url)
  outerpage_soup = BeautifulSoup(outerpage_request.content, 'html.parser')

  articles2 = outerpage_soup.find_all('h3')
  start = time.time()
  book_extracted = 0

  for book in articles2:
    page_page = book.find('a')['href']
    page_request = requests.get(url1+page_page)
    page_soup = BeautifulSoup(page_request.content, 'html.parser')

    book_title = page_soup.find('h1').text
    book_price = page_soup.find('p', class_='price_color').text.strip()
    book_rating = page_soup.find('p', class_='star-rating')['class'][1]
    book_genre = page_soup.find('ul', class_='breadcrumb').find_all('a')[2].text.strip()
    book_stock = page_soup.find('p', class_='instock availability').text.strip()

    end = time.time()
    total_time = (end-start)/60.0

    # append into empty list
    books_perpage.append([book_title, book_price, book_stock, book_genre, book_rating])

  # print(book_title)
  # print(book_price)
  # print(book_rating)
  # print(book_genre)
  # print(book_stock)

# add to dataframe
pd.DataFrame(books_perpage, columns=['Title', 'Price', 'Stock', 'Genre', 'Rating'])


,Title,Price,Stock,Genre,Rating
0,A Light in the Attic,£51.77,In stock (22 available),Poetry,Three
1,Tipping the Velvet,£53.74,In stock (20 available),Historical Fiction,One
2,Soumission,£50.10,In stock (20 available),Fiction,One
3,Sharp Objects,£47.82,In stock (20 available),Mystery,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock (20 available),History,Five
...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock (1 available),Classics,One
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock (1 available),Sequential Art,Four
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock (1 available),Historical Fiction,Five
998,1st to Die (Women's Murder Club #1),£53.98,In stock (1 available),Mystery,One


In [96]:
# save scraped data to excel
pd.DataFrame(books_perpage, columns=['Title', 'Price', 'Stock', 'Genre', 'Rating']).to_excel('BooksToCrape01.xlsx',index=False)

In [8]:
for i in articles:
  # Method #1
  # title = i.find('img').attrs['alt']
  # star = i.find('p')['class'][1]
  # price = i.find('p', class_='price_color').text

  # Method #2
  title = i.find('h3').text
  star = i.find('p',class_='star-rating')['class'][1]
  price = i.find('p', class_='price_color').text.strip()
  stock = i.find('p', class_='instock availability').text.strip()
  genre = i.find('li', class_='active')

  # print(title)
  # print(star)
  # print(price)
  print(genre)
  # print(stock)


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


##### Extract book per page
(take less time because it scrape per page only)

In [112]:
books = []
for j in range(1,51):
  page_url = f'https://books.toscrape.com/catalogue/page-{j}.html'
  page_request = requests.get(page_url)
  page_request = page_request.content

  # parse html content
  page_soup = BeautifulSoup(page_request,'html.parser')

  page_list = page_soup.find('ol')
  page_article = page_list.find_all('article', class_='product_pod')
  for page in page_article:
    book_title = page.find('img').attrs['alt']
    book_rating = page.find('p')['class'][1]
    book_price = page.find('p', class_='price_color').text
    book_stock = page.find('p', class_='instock availability').text.strip()

    books.append([book_title, book_price, book_rating, book_stock])
# print(books)
pd.DataFrame(books, columns=['Title','Price','Ratings', 'Stock'])

,Title,Price,Ratings,Stock
0,A Light in the Attic,£51.77,Three,In stock
1,Tipping the Velvet,£53.74,One,In stock
2,Soumission,£50.10,One,In stock
3,Sharp Objects,£47.82,Four,In stock
4,Sapiens: A Brief History of Humankind,£54.23,Five,In stock
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,One,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,Four,In stock
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,Five,In stock
998,1st to Die (Women's Murder Club #1),£53.98,One,In stock


In [114]:
# save scraped data to excel
pd.DataFrame(books, columns=['Title','Price','Ratings', 'Stock']).to_excel('BooksToCrape02.xlsx',index=False)

In [120]:
b1 = 'BooksToCrape01'
b2 = 'BooksToCrape02'

# load scraped file
df_01 = pd.read_excel(rf'C:\Users\wareh\Documents\VS Code Folders\Portfolio Project\{b1}.xlsx')

In [130]:
df_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   1000 non-null   object
 1   Price   1000 non-null   object
 2   Stock   1000 non-null   object
 3   Genre   1000 non-null   object
 4   Rating  1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [142]:
df_01_05 = df_01.groupby('Genre').count().reset_index().sort_values(by='Title', ascending=False) [:5]
df_01_05

,Genre,Title,Price,Stock,Rating
14,Default,152,152,152,152
28,Nonfiction,110,110,110,110
41,Sequential Art,75,75,75,75
1,Add a comment,67,67,67,67
17,Fiction,65,65,65,65


#### <hr>

### 

### 